In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

# pull feature engineering notebook
# add your own feature engineering functions to features.py and import them
# only rule is the function must take a dataframe and return a dataframe (with your new features)
from features import engineer, scrub, no_photo, num_keyword

color = sns.color_palette()
%matplotlib inline

df_train = pd.read_json("train.json")
df_test = pd.read_json("test.json")

### PRE-PROCESSING

In [2]:
#engineer features (from script)
scrub_and_engineer = [scrub, engineer, no_photo, num_keyword]
for func in scrub_and_engineer:
    df_train = func(df_train)
    df_test = func(df_test)

In [59]:
[x for x in df_train.columns.tolist() if df_train[x].dtype != 'O'] 

[u'bathrooms',
 u'bedrooms',
 u'created',
 u'latitude',
 u'listing_id',
 u'longitude',
 u'price',
 'num_photos',
 'num_features',
 'num_description_words',
 'created_year',
 'created_month',
 'created_day',
 'n_no_photo',
 'n_num_keyfeat_score']

### Hyperparameters

In [43]:
# Cross val
test_size=0.20

# Random Forest
n_estimators=1000

### Baseline training

In [4]:
feats_to_train = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day"]
X = df_train[feats_to_train]
y = df_train["interest_level"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size)

clf = RandomForestClassifier(n_estimators=n_estimators)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
baseline_logloss = log_loss(y_val, y_val_pred)

In [6]:
baseline_logloss

0.62263398023396599

### Training the new model

In [ ]:
# feats_to_train = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
#              "num_photos", "num_features", "num_description_words",
#              "created_year", "created_month", "created_day", 'n_num_keyfeat_score', 'n_no_photo']
feats_to_train = [x for x in df_train.columns.tolist() if df_train[x].dtype not in ['O', '<M8[ns]']] 
X = df_train[feats_to_train]
y = df_train["interest_level"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size)

clf = RandomForestClassifier(n_estimators=n_estimators)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
test_logloss = log_loss(y_val, y_val_pred)

In [ ]:
test_logloss

### Testing for raw improvement

In [61]:
if baseline_logloss - test_logloss > 0:
    print "Model improved, save and submit"
else:
    print "Use baseline model, did not improve"

Use baseline model, did not improve


### Clean up and output a submission file

In [47]:
def output(df_test, clf):
    X = df_test[feats_to_train]
    y = clf.predict_proba(X)
    labels2idx = {label: i for i, label in enumerate(clf.classes_)}
    sub = pd.DataFrame()
    sub["listing_id"] = df_test["listing_id"]
    for label in ["high", "medium", "low"]:
        sub[label] = y[:, labels2idx[label]]
    return sub

In [ ]:
sub = output(df_test, clf)

In [ ]:
sub.to_csv("submission_rf.csv", index=False)

In [ ]:
submission = pd.read_csv('submission_rf.csv')

In [ ]:
len(submission)